In [1]:
import numpy as np
from random import randint
import pandas as pd

In [8]:
def weightedRandom(row, tpath):
    x = randint(0, (tpath*25))
    n = ((tpath*25)-row['fitness'])*1.2
    if x <= n:
        return 1
    else:
        return 0

def evalFit(df, path):
    finalFitness = 0
    path = path.values[0]
    for i, val in enumerate(path):
        if i < len(path)-1:
            currentRow = df.loc[(df['from_id'] == path[i]) & (df['to_id'] == path[i+1])]
            print(currentRow['car_m_t'].tolist()[0])
            finalFitness += currentRow['car_m_t'].tolist()[0]
    return finalFitness


def parentSelection(df, n, path):
    df['pass'] = df.apply(weightedRandom,tpath=len(path), axis=1)
    if(len(df['sequence']) > n):
        val = df.iloc[0].fitness
        df = df[~(df['fitness'] >= val+10)]
    if(len(df) > 1):
        df = df.drop_duplicates(subset=['sequence'])
    df = df.sort_values(by=['fitness']).reset_index(drop=True)
    return df

def selectPassed(df):
    df = df.loc[df['pass'] == 1]
    n1, n2 = randint(0, len(df['sequence'])-1), randint(0, len(df['sequence'])-1)
    pass1 = df['sequence'].iloc[n1]
    pass2 = df['sequence'].iloc[n2]
    return pass1, pass2

def crossover(df, iValList):
    
    df = df.reset_index(drop=True)
    return df

def mutation(df, iValList):
    
    df = df.reset_index(drop=True)
    return df

def GA(iValDF, pathList):
    populationSize = 20
    resultsOverTimeList = []
    df = pd.DataFrame(columns=['sequence', 'fitness'])
    tempfit = 0
    df['sequence'] = [pathList]
    df['fitness'] = evalFit(iValDF, df['sequence'])
    df = parentSelection(df, populationSize, path)
    print("Starting iterations... Randomized population best fitness:")
    print(df.loc[0])
    for i in np.arange(100):
        df = crossover(df, iValDF)
        df = parentSelection(df, populationSize,path)
        if i%100 == 0 and i != 0:
            if tempfit == df['fitness'].iloc[0]:
                df = mutation(df, iValDF)
            else:
                tempfit = df['fitness'].iloc[0]
            #print(i, " iterations completed. Current best fitness:")
            #print(df.loc[0])
        resultsOverTimeList.append(df['fitness'].loc[0])
    print("All iterations completed. Best fitness achieved:")
    print(df.loc[0])
    return df, resultsOverTimeList